# tdms原始数据读取 
## 不要随便运行， 注释掉了，要用再解除注释

In [ ]:
# import sys
# from pathlib import Path

# PROJECT_ROOT = Path("/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/High_similar_ML/data_process")
# sys.path.insert(0, str(PROJECT_ROOT))

# print("📌 Project root set to:", PROJECT_ROOT)


# from my_lib.KAIST.KAIST_utils import process_kaist_file
# from pathlib import Path

# # =========================
# # 1. 原始 TDMS 文件列表
# # =========================
# tdms_files = [
#     "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_BPFI_03.tdms",
#     "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_BPFI_10.tdms",
#     "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_BPFI_30.tdms",

#     "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_BPFO_03.tdms",
#     "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_BPFO_10.tdms",
#     "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_BPFO_30.tdms",

#     "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_Misalign_01.tdms",
#     "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_Misalign_03.tdms",
#     "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_Misalign_05.tdms",

#     "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_Normal.tdms",

#     "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_Unbalance_0583mg.tdms",
#     "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_Unbalance_1169mg.tdms",
#     "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_Unbalance_1751mg.tdms",
#     "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_Unbalance_2239mg.tdms",
#     "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_Unbalance_3318mg.tdms",
# ]


# # =========================
# # 2. 保存目录（Linux 侧）
# # =========================
# save_dir = Path("/home/charles/HZU/Data_processed/HSML/KAIST/current&temp_raw")
# save_dir.mkdir(parents=True, exist_ok=True)

# # =========================
# # 3. 批量处理
# # =========================
# saved_csv_files = []

# for tdms_path in tdms_files:
#     print(f"🚀 Processing: {tdms_path}")
#     out_csv = process_kaist_file(
#         file_path=tdms_path,
#         save_dir=str(save_dir),
#         data_type="temperature_current"
#     )
#     saved_csv_files.append(out_csv)
#     print(f"✅ Saved to: {out_csv}")

# print(f"\n🎉 Done! Total files processed: {len(saved_csv_files)}")



## 重写列名

In [ ]:
from pathlib import Path
import pandas as pd

# =========================
# 1. 输入 / 输出目录
# =========================
src_dir = Path("/home/charles/HZU/Data_processed/HSML/KAIST/current&temp_raw")
dst_dir = Path("/home/charles/HZU/Data_processed/HSML/KAIST/current&temp_raw_withlabel")
dst_dir.mkdir(parents=True, exist_ok=True)

# =========================
# 2. 原始 → 标准列名映射（核心）
# =========================
column_rename_map = {
    "timestamp": "timestamp",

    "cDAQ9185-1F486B5Mod1/ai0": "Temp_A(°C)",
    "cDAQ9185-1F486B5Mod1/ai1": "Temp_B(°C)",

    "cDAQ9185-1F486B5Mod2/ai0": "I_U(A)",
    "cDAQ9185-1F486B5Mod2/ai2": "I_V(A)",
    "cDAQ9185-1F486B5Mod2/ai3": "I_W(A)",
}

expected_cols = set(column_rename_map.values())

# =========================
# 3. 批量处理
# =========================
csv_files = sorted(src_dir.glob("*.csv"))
print(f"📂 Found {len(csv_files)} CSV files")

for csv_path in csv_files:
    print(f"\n🔄 Processing: {csv_path.name}")

    df = pd.read_csv(csv_path)

    # 校验：是否包含所有原始列
    missing_raw = set(column_rename_map.keys()) - set(df.columns)
    if missing_raw:
        raise ValueError(
            f"{csv_path.name} 缺少原始列: {missing_raw}"
        )

    # 重命名
    df = df.rename(columns=column_rename_map)

    # 校验：重命名后是否齐全
    missing_after = expected_cols - set(df.columns)
    if missing_after:
        raise ValueError(
            f"{csv_path.name} 重命名后缺少列: {missing_after}"
        )

    # 保存
    out_path = dst_dir / csv_path.name
    df.to_csv(out_path, index=False)

    print(f"✅ Saved to: {out_path}")

print("\n🎉 所有文件列名已统一并成功保存。")


## 加入标签

In [6]:
from pathlib import Path
import pandas as pd

# =========================
# 1. 数据目录
# =========================
data_dir = Path("/home/charles/HZU/Data_processed/HSML/KAIST/current&temp_raw_withlabel")

# =========================
# 2. 文件名 → 标签 映射表
# =========================
label_map = {
    "0Nm_Normal": 1,

    "0Nm_BPFI_03": 2,
    "0Nm_BPFI_10": 3,
    "0Nm_BPFI_30": 4,

    "0Nm_BPFO_03": 5,
    "0Nm_BPFO_10": 6,
    "0Nm_BPFO_30": 7,

    "0Nm_Misalign_01": 8,
    "0Nm_Misalign_03": 9,
    "0Nm_Misalign_05": 10,

    "0Nm_Unbalance_0583mg": 11,
    "0Nm_Unbalance_1169mg": 12,
    "0Nm_Unbalance_1751mg": 13,
    "0Nm_Unbalance_2239mg": 14,
    "0Nm_Unbalance_3318mg": 15,
}

# =========================
# 3. 批量处理
# =========================
csv_files = sorted(data_dir.glob("*.csv"))
print(f"📂 Found {len(csv_files)} CSV files")

for csv_path in csv_files:
    stem = csv_path.stem   # 去掉 .csv
    print(f"\n🔄 Processing: {csv_path.name}")

    if stem not in label_map:
        raise ValueError(f"❌ 文件名未在映射表中: {csv_path.name}")

    label = label_map[stem]

    df = pd.read_csv(csv_path)

    # 新增 label 列（放在最后，最稳）
    df["label"] = label

    # 覆盖保存
    df.to_csv(csv_path, index=False)

    print(f"✅ Added label={label} to {csv_path.name}")

print("\n🎉 所有文件已成功添加 label 列。")


📂 Found 15 CSV files

🔄 Processing: 0Nm_BPFI_03.csv
✅ Added label=2 to 0Nm_BPFI_03.csv

🔄 Processing: 0Nm_BPFI_10.csv
✅ Added label=3 to 0Nm_BPFI_10.csv

🔄 Processing: 0Nm_BPFI_30.csv
✅ Added label=4 to 0Nm_BPFI_30.csv

🔄 Processing: 0Nm_BPFO_03.csv
✅ Added label=5 to 0Nm_BPFO_03.csv

🔄 Processing: 0Nm_BPFO_10.csv
✅ Added label=6 to 0Nm_BPFO_10.csv

🔄 Processing: 0Nm_BPFO_30.csv
✅ Added label=7 to 0Nm_BPFO_30.csv

🔄 Processing: 0Nm_Misalign_01.csv
✅ Added label=8 to 0Nm_Misalign_01.csv

🔄 Processing: 0Nm_Misalign_03.csv
✅ Added label=9 to 0Nm_Misalign_03.csv

🔄 Processing: 0Nm_Misalign_05.csv
✅ Added label=10 to 0Nm_Misalign_05.csv

🔄 Processing: 0Nm_Normal.csv
✅ Added label=1 to 0Nm_Normal.csv

🔄 Processing: 0Nm_Unbalance_0583mg.csv
✅ Added label=11 to 0Nm_Unbalance_0583mg.csv

🔄 Processing: 0Nm_Unbalance_1169mg.csv
✅ Added label=12 to 0Nm_Unbalance_1169mg.csv

🔄 Processing: 0Nm_Unbalance_1751mg.csv
✅ Added label=13 to 0Nm_Unbalance_1751mg.csv

🔄 Processing: 0Nm_Unbalance_2239mg.csv
✅